# UWF22 dataset stats

This notebook explores the data from the UFW22Zeek database

In this notebook, we explore:
- Basic dataset stats


In [ ]:
from noether.datasets.UWF22_local import UWF22L
from ordered_set import OrderedSet

dataset = UWF22L("/data/datasets/UWF22")

dataset.prepare_data()
dataset.setup(stage="fit")

In [6]:
next(iter(dataset.generate_batches()))

/data/Try1/noether/datasets/UFW22_local.py:254: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  .apply(lambda x: (x[0], x[1]), axis=1)) # type: ignore
/data/Try1/noether/datasets/UFW22_local.py:258: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  .apply(lambda x: (x[0], x[1]), axis=1)) # type: ignore


KeyError: None

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import clear_output

def UFW22_unique_v1(dataset, show: bool = False):
    # Get dataset columns
    columns = list(next(iter(dataset.generate_bins())).columns)

    # Generate keymaps
    exclude_columns = [
        "time",
        "source computer",
        "destination computer",
        "source user@domain",
        "destination user@domain"
    ]
    keyword_map = {col: OrderedSet([]) for col in columns if col not in exclude_columns}
    hostmap = OrderedSet([])
    usermap = OrderedSet([])

    # Keep track of the added values per column
    added_items_per_bin = {col: [] for col in columns if col not in exclude_columns}
    added_items_per_bin["hosts"] = []
    added_items_per_bin["services"] = []

    def count_updates(ordered_set: OrderedSet, update):
        set_size = len(ordered_set)
        ordered_set.update(update)
        new_size = len(ordered_set)
        return new_size

    def show_plot(added_items_per_bin):
        if show == False:
            return
        
        df = pd.DataFrame(added_items_per_bin)

        clear_output(wait=True)
        df.index = df.index/dataset.batch_size
        ax = df[["hosts"]].plot(xlabel="batch", ylabel="Unique count")
        plt.grid(True)
        plt.show()

    for batch_id, (batch, stage) in enumerate(dataset.generate_batches()):
        # Go trough every bin
        for bin in batch:
            # Update keyword map
            for col in keyword_map.keys():
                diff = count_updates(keyword_map[col], bin[col].dropna().unique())
                added_items_per_bin[col].append(diff)
            
            diff1 = count_updates(hostmap, bin["source computer"].dropna().unique())
            diff2 = count_updates(hostmap, bin["destination computer"].dropna().unique())
            added_items_per_bin["hosts"].append(diff1 + diff2)

            diff1 = count_updates(usermap, bin["source user@domain"].dropna().unique())
            diff2 = count_updates(usermap, bin["destination user@domain"].dropna().unique())
            added_items_per_bin["services"].append(diff1 + diff2)

        show_plot(added_items_per_bin)


    show_plot(added_items_per_bin)
    df = pd.DataFrame(added_items_per_bin)
    df.to_parquet("keywordmap_percent.parquet")

LANL_unique_v1(dataset, show=True)

### Dataset stats

In [ ]:
import pandas as pd
from ordered_set import OrderedSet

set = OrderedSet([])

df = pd.read_parquet("/data/datasets/UWF22/7.parquet")
set.update(df[["src_ip_zeek", "src_port_zeek"]].drop_duplicates().reset_index(drop=True).apply(lambda x: (x[0], x[1]), axis=1))
set

In [ ]:
df["service_id"] = df[["src_ip_zeek", "src_port_zeek"]].apply(lambda x: (x[0], x[1]), axis=1).map(set.index)